In [19]:
## %config Completer.use_jedi = False 
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
import sklearn as skl
import sklearn.preprocessing as skl_pre
import sklearn.linear_model as skl_lin
from keras.models import Sequential
from keras.layers import Flatten, Dense, InputLayer
from keras.losses import SparseCategoricalCrossentropy
from keras.utils import np_utils
import sklearn.mixture as skl_mix
from utils import mnist_reader
from utils import helper
import copy
from keras.datasets import mnist
from utils import helper

## Load In Data ##
X_train, y_train = mnist_reader.load_mnist('data/fashion', kind='train')
X_test, y_test = mnist_reader.load_mnist('data/fashion', kind='t10k')


## Standardize Features to Standard Gaussians ##
scaler = skl_pre.StandardScaler()
X_train = scaler.fit_transform(X_train)
# Note we use the same transformation on the test set.
X_test = scaler.transform(X_test)

X_train_28x28 = helper.vector_to_matrix_mnist(X_train)
X_test_28x28 = helper.vector_to_matrix_mnist(X_test)



In [20]:
## UNUSED CODE ##
## My original intention was to initialize centers by Lloyd's using a distance metric that encourages evenly separated clusters,
## then apply X-means, then pass into an EM algorithm.
## It turns out, scikit-learn's GMM model does k-means with random initialization, then applies their optimized, built in EM algorithm.


n = X_train.shape[1]
d = X_train.shape[0]
## greedily compute the next k-1 centers furthest centers given a starting center mu, init
## the data X, and k-clusters desired

## distance measures to separate clusters
## Try using sqrt of L2
def dist(x1, x2):
    return np.sqrt(np.linalg.norm(x1 - x2))
## just use built in GMM model
## We hope to maximize the iterative averaged distance
def furthest_centers(mu, init, X, k):
    np.random.seed(1999) ## 
    init = np.random.randint(0,n) ## get random point in dataset
    visited_idx = [init] ## keep track of indices
    centers = [X[init]]
    for j in range(k-1):
        for i in range(n):
            if i in visited_idx:
                continue ## one downside is that the norm only interprets total difference of pixels, and so the clusterings may be poor.
            currmax = float('-inf')
            avg_dist = 0
            for c_j in centers: ## can we come up with a measure that encourages points far from all centers?
                avg_dist += dist(c_j, X[i])
            avg_dist /= len(centers)
            if avg_dist > currmax:
                currmax = avg_dist
                maxidx = i
        centers.append(X[maxidx])
        visited_idx.append(maxidx)
    return centers

In [21]:
k_clusters = 15 ## do more clusters than likely necessary.
## Cluster by k-means
GMM = skl_mix.GaussianMixture(k_clusters, 'full', random_state = 1999, verbose = True)
GMM.fit(X_train)

C:\Users\dk\anaconda3\lib\site-packages\sklearn\utils\validation.py:68: FutureWarning: Pass covariance_type=full as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


Initialization 0
  Iteration 10
  Iteration 20
  Iteration 30
  Iteration 40
Initialization converged: True


GaussianMixture(n_components=15, random_state=1999, verbose=True)

Training data split: For each model m_i with a corresponding cluster c_i, we first give it all the data in c_i, then bag/resample the remaining, 80% data from main clustering, 20% from all clusters (inclusive).

Idea: Resample and train using the generated gaussians.

In [22]:
Clusterings = [] ## store the indices of the clusterings
pred = GMM.predict(X_train)

In [23]:
X_data_clusters = []
Y_data_clusters = []
for i in range(k_clusters):
    Clusterings.append(np.where(pred == i))
    remaining = X_train.shape[0] - Clusterings[0][0].shape[0]
    main_cluster_int = int(np.floor(0.8*(remaining)))
    outer_cluster_int = remaining - main_cluster_int
    main = np.random.choice(Clusterings[0][0],main_cluster_int)
    outer = np.random.choice(np.arange(0, X_train.shape[0]), outer_cluster_int)
    X_i = np.concatenate([X_train_28x28[Clusterings[0][0]],X_train_28x28[main], X_train_28x28[outer]], axis = 0)
    Y_i = np.concatenate([y_train[Clusterings[0][0]],y_train[main], y_train[outer]], axis = 0)
    X_i = X_i[...,None]
    X_data_clusters.append(X_i)
    Y_data_clusters.append(Y_i)

In [24]:
## Do LeNet-5 Architecture for EACH data set
##~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~##
## LeNet-5 Architecture keras code is taken from RPI FALL 2020 CSCI 4961 - Machine Learning & Optimization notes, by Prof. Alex Gittens ##
## Note that the Architecture itself is from "Gradient Based Learning Applied to Document Recognition", (LeCun et al., 1998)            ##
##~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~##
from keras.models import Sequential
from keras.layers import Conv2D, AveragePooling2D, Flatten, Dense, InputLayer
from keras.losses import SparseCategoricalCrossentropy
from keras.utils import np_utils

models = []
for m in range(k_clusters):
    lenet = Sequential([
    InputLayer(input_shape=(28, 28, 1)),
    Conv2D(6, kernel_size=(5,5), strides=(1,1), activation='tanh', padding="same", name="C1"),
    AveragePooling2D(pool_size=(2,2), strides=(1,1), padding='valid', name="A1"), # no padding before pooling,
    Conv2D(16, kernel_size=(5,5), strides=(1,1), activation='tanh', name="C2"), # by default padding is "valid",
    AveragePooling2D(pool_size=(2,2), strides=(2,2), padding='valid', name="A2"),
    Conv2D(120, kernel_size=(5,5), strides=(1,1), activation='tanh', padding='valid', name="C3"),
    Flatten(name="F"),
    Dense(84, activation='tanh', name="D1"),
    Dense(10, activation='softmax', name="D2")])
    lenet.compile(loss=SparseCategoricalCrossentropy(), optimizer='adam', metrics=['accuracy'])
    models.append(lenet)
    

In [25]:
hist = []
for m in range(k_clusters):
    history = (models[m]).fit(X_data_clusters[m], Y_data_clusters[m], epochs=10, batch_size=128, verbose=1)
    hist.append(history)

Epoch 1/10
469/469 [==============================] - 34s 71ms/step - loss: 0.2892 - accuracy: 0.9049
Epoch 2/10
469/469 [==============================] - 30s 65ms/step - loss: 0.0946 - accuracy: 0.9678
Epoch 3/10
469/469 [==============================] - 31s 65ms/step - loss: 0.0825 - accuracy: 0.9700
Epoch 4/10
469/469 [==============================] - 32s 68ms/step - loss: 0.0776 - accuracy: 0.9723
Epoch 5/10
469/469 [==============================] - 31s 66ms/step - loss: 0.0737 - accuracy: 0.9739
Epoch 6/10
469/469 [==============================] - 30s 64ms/step - loss: 0.0696 - accuracy: 0.9747
Epoch 7/10
469/469 [==============================] - 30s 64ms/step - loss: 0.0681 - accuracy: 0.9749
Epoch 8/10
469/469 [==============================] - 30s 64ms/step - loss: 0.0667 - accuracy: 0.9764
Epoch 9/10
469/469 [==============================] - 30s 63ms/step - loss: 0.0683 - accuracy: 0.9748
Epoch 10/10
469/469 [==============================] - 30s 63ms/step - loss: 0.065

469/469 [==============================] - 35s 75ms/step - loss: 0.0734 - accuracy: 0.9734
Epoch 1/10
469/469 [==============================] - 39s 82ms/step - loss: 0.3144 - accuracy: 0.9035
Epoch 2/10
469/469 [==============================] - 38s 81ms/step - loss: 0.1007 - accuracy: 0.9662
Epoch 3/10
469/469 [==============================] - 36s 77ms/step - loss: 0.0834 - accuracy: 0.9709
Epoch 4/10
469/469 [==============================] - 36s 76ms/step - loss: 0.0744 - accuracy: 0.9739
Epoch 5/10
469/469 [==============================] - 37s 80ms/step - loss: 0.0705 - accuracy: 0.9748
Epoch 6/10
469/469 [==============================] - 38s 80ms/step - loss: 0.0663 - accuracy: 0.9771
Epoch 7/10
469/469 [==============================] - 39s 84ms/step - loss: 0.0655 - accuracy: 0.9766
Epoch 8/10
469/469 [==============================] - 40s 85ms/step - loss: 0.0625 - accuracy: 0.9785
Epoch 9/10
469/469 [==============================] - 36s 77ms/step - loss: 0.0627 - accuracy

In [44]:
X_test, y_test = mnist_reader.load_mnist('data/fashion', kind='t10k')
X_test = scaler.transform(X_test)
X_test_28x28 = helper.vector_to_matrix_mnist(X_test)

test_pred = np.zeros([X_test.shape[0], 10]) ## 10 for 10 classes in fashion mnist
GMM_test = GMM.predict_proba(X_test) ## 10,000 x 15
X_test_28x28 = X_test_28x28[..., None]
for m in range(k_clusters):
    test_probabilities = (models[m]).predict(X_test_28x28) ## 10000 x 10?
    test_pred += np.diag(GMM_test[:,m])@test_probabilities ## scale our predictions by our gaussians
Y_test_pred = np.argmax(test_pred, axis = 1) ## get the class!
error = np.mean(np.where(y_test - Y_test_pred != 0, 1, 0)) ## different classes
print(error)

0.1744


In [52]:
X_train_28x28x1 = X_train_28x28[...,None]
lenet_full = Sequential([
    InputLayer(input_shape=(28, 28, 1)),
    # padding="same" pads input with enough zeros in each direction before convolution that the result of the convolution has the same size as the input image
    Conv2D(6, kernel_size=(5,5), strides=(1,1), activation='tanh', padding="same", name="C1"),
    AveragePooling2D(pool_size=(2,2), strides=(1,1), padding='valid', name="A1"), # no padding before pooling,
    Conv2D(16, kernel_size=(5,5), strides=(1,1), activation='tanh', name="C2"), # by default padding is "valid",
    AveragePooling2D(pool_size=(2,2), strides=(2,2), padding='valid', name="A2"),
    Conv2D(120, kernel_size=(5,5), strides=(1,1), activation='tanh', padding='valid', name="C3"),
    Flatten(name="F"),
    Dense(84, activation='tanh', name="D1"),
    Dense(10, activation='softmax', name="D2")
 ])
lenet_full.compile(loss=SparseCategoricalCrossentropy(), optimizer='adam', metrics=['accuracy'])
history = lenet_full.fit(X_train_28x28x1, y_train, epochs=10, batch_size=128, verbose=1)


Epoch 1/10
469/469 [==============================] - 36s 75ms/step - loss: 0.5864 - accuracy: 0.7873
Epoch 2/10
469/469 [==============================] - 34s 72ms/step - loss: 0.3664 - accuracy: 0.8697
Epoch 3/10
469/469 [==============================] - 35s 75ms/step - loss: 0.3426 - accuracy: 0.8777
Epoch 4/10
469/469 [==============================] - 34s 72ms/step - loss: 0.3330 - accuracy: 0.8794
Epoch 5/10
469/469 [==============================] - 36s 77ms/step - loss: 0.3084 - accuracy: 0.8904
Epoch 6/10
469/469 [==============================] - 35s 74ms/step - loss: 0.3112 - accuracy: 0.8884
Epoch 7/10
469/469 [==============================] - 37s 78ms/step - loss: 0.3032 - accuracy: 0.8908
Epoch 8/10
469/469 [==============================] - 36s 77ms/step - loss: 0.3071 - accuracy: 0.8879
Epoch 9/10
469/469 [==============================] - 35s 75ms/step - loss: 0.2979 - accuracy: 0.8932
Epoch 10/10
469/469 [==============================] - 34s 73ms/step - loss: 0.291

In [57]:
test_probabilities = lenet_full.predict(X_test_28x28) ## 10000 x 10?
Y_test_pred = np.argmax(test_probabilities, axis = 1) ## get the class
error = np.mean(np.where(y_test - Y_test_pred != 0, 1, 0)) ## different classes
print(error)

0.1286


TO DO:
Double check LeNet5 with the full data set. Accuracy seems significantly off.
** COULD BE BECAUSE OF DATA SCALING TO STD GAUSSIANS.
- Why do we get decreased accuracy? 